# Content Clean Up

In [12]:
from app.repositories.LocalFolderRepo import LocalFolderRepo
from app.repositories.HashedContentStorage import HashedContentStorage, HashedContentStatus
from app.services.ContentService import ContentService

local_folder = LocalFolderRepo("./hashed_files")
hashed_content_storage = HashedContentStorage(local_folder)
content_service = ContentService(hashed_content_storage)

In [13]:
hash = '4c1299d6bb20b61c4594292009d5aad77bdee147273a71cdd8616a2696f2218d'
raw_md = content_service.get_markdown(hash)
assert raw_md is not None, "Content not found"
print(len(raw_md))

30554


In [ ]:
from app.utils.MarkdownChunker import MarkdownChunker

chunker = MarkdownChunker()
chunks = chunker.chunk(raw_md)
print(f"Number of chunks: {len(chunks)}")

Tesla, Inc. designs, develops, manufactures, sells, and leases fully electric vehicles and energy-
generation and storage systems, and offers services related to its products, such as leasing of
electric vehicles and sales of automotive regulatory credits.  Its automotive products include
the Model 3, Model Y, Model S, and Model X. Currently, the company produces its electric cars
at gigafactories in the United States, Germany, and China.
Investment Overview

Leading  EV  manufacturer  backed  by  economic  MOATs.  Tesla  is  a  leading  global  EV
manufacturer,  backed  by  its  firm  market  leadership  with  an  estimated  20%  market  share
based on global sales, and healthy automotive margins seen in the industry (FY23 at 17%, on
top end range of peers between 10-20%) which we believe is a commendable feat given that
many legacy OEMs are still witnessing losses in EV production. Tesla's leading market share is
backed by its economic MOAT in EV charging infrastructure and superchar